In [ ]:
import pandas as pd
import sklearn 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("train.csv")

In [ ]:
df.Sex.unique()

In [ ]:
df['Sex_binary'] = np.where(df['Sex'] == "female", 1, 0)
df.drop(columns='Sex', inplace=True)

In [ ]:
# set Embarked_ternary to 1 for C, 2 for Q, 3 for S
df['Embarked_ternary'] = np.where(df["Embarked"] == "C", 1, 
         (np.where(df["Embarked"] == "Q", 2, 3)))


In [ ]:
df

In [ ]:
count_nan = len(df['Cabin']) - df['Cabin'].count()


In [ ]:
count_nan

OK, so most of the Cabin values are NaN, so we can probably drop this column.

In [ ]:
df.drop(columns='Cabin', inplace=True)

Not sure about name column, might be useful data in len(name)?

In [ ]:
df

In [ ]:
# Slightly more likely to survive if younger
df.groupby(['Survived']).Age.mean()


In [ ]:
df.groupby(['Survived']).Fare.mean()


In [ ]:
# who embarked at Cherbourg?
print("number embarked at Cherbourg", len(df.loc[df['Embarked_ternary'] == 1]))

print("average survival status", df.loc[df['Embarked_ternary'] == 1].Survived.mean())


In [ ]:
# who embarked at Queenstown?

print("number embarked at Queenstown", len(df.loc[df['Embarked_ternary'] == 2]))

print("average survival status", df.loc[df['Embarked_ternary'] == 2].Survived.mean())


In [ ]:
# who embarked at Southampton?
print("number embarked at Southampton", len(df.loc[df['Embarked_ternary'] == 3]))

print("average survival status", df.loc[df['Embarked_ternary'] == 3].Survived.mean())


OK, so there is some evidence that embarking at Cherbourg is correlated with survival. So let's turn embarkation_ternary into a binary condition: emarbarked_at_cherbourg

In [ ]:
df['Embarked_at_cherbourg'] = np.where(df['Embarked_ternary'] == 1, 1, 0)

In [ ]:
df.dtypes

In [ ]:
df.drop(columns="Embarked_ternary", inplace=True)

In [ ]:
# how do the name lengths vary? And are they correlated with survival?

names_lens = [(name[1], len(name[1])) for name in df["Name"].iteritems()]
name_lens = [name_len[1] for name_len in names_lens]
plt.hist(name_lens)
plt.show()


In [ ]:
# add an int value for name length
df['Name_len'] = [len(name[1]) for name in df["Name"].iteritems()]


In [ ]:
df.corr()

In [ ]:
# woah, having a long name is almost as highly correlated with survival as passanger class!

In [ ]:
# todo: engineer ticket class
# for now, we'll just drop it

df.drop(columns=['PassengerId', 'Embarked', 'Name', 'Ticket'], inplace=True)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.isnull().any()

We have some null ages. Let's see how many.

In [ ]:
len(df.loc[df.Age.isnull()])

And how are they distributed?

In [ ]:
# naively fill nulls for now
df.fillna(df.mean(), inplace=True)

Scale data


In [ ]:
df

In [ ]:


(train_passengers, train_labels) = (df.drop('Survived', axis=1), df['Survived'])




In [ ]:
df

In [ ]:

from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(train_passengers)
scaler.transform(train_passengers, copy=False)

In [ ]:
# urgh, standardscaler returns a numpy array, not df

# scaler.transform(train_passengers, copy=False) let's try something else

In [ ]:

    # train_passengers is df
train_passengers[train_passengers.columns] = scaler.fit_transform(train_passengers[train_passengers.columns])

In [ ]:
# make a function with preceding data-wrangling steps, so that we can perform the same operations on the test set
def wrangle(df):
    """
    takes a df with same format as training set.
    returns df in same format as modified training set
    """
    df['Sex_binary'] = np.where(df['Sex'] == "female", 1, 0)
    df.drop(columns='Sex', inplace=True)
    df['Embarked_ternary'] = np.where(df["Embarked"] == "C", 1, 
         (np.where(df["Embarked"] == "Q", 2, 3)))
    df.drop(columns='Cabin', inplace=True)
    df['Embarked_at_cherbourg'] = np.where(df['Embarked_ternary'] == 1, 1, 0)
    df.drop(columns="Embarked_ternary", inplace=True)
    names_lens = [(name[1], len(name[1])) for name in df["Name"].iteritems()]
    name_lens = [name_len[1] for name_len in names_lens]
    df['Name_len'] = [len(name[1]) for name in df["Name"].iteritems()]    
    df.drop(columns=['PassengerId', 'Embarked', 'Name', 'Ticket'], inplace=True)
    
    # naively fill in nulls for now
    df.fillna(df.mean(), inplace=True)
    return df

In [ ]:
# load and transform test data
df_test = pd.read_csv("test.csv")
df_test = wrangle(df_test)

In [ ]:
import keras
from keras import models 
from keras import layers
from keras import regularizers

In [ ]:


test_passengers = df_test


In [ ]:
test_passengers[test_passengers.columns] = scaler.fit_transform(test_passengers[test_passengers.columns])

In [ ]:
train_passengers.shape

In [ ]:
network = models.Sequential()
network.add(layers.Dense(714, activation='relu', input_shape=(8,))) # experiment with different activation functions

keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True, beta_initializer='zeros', gamma_initializer='ones', moving_mean_initializer='zeros', moving_variance_initializer='ones', beta_regularizer=None, gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)
network.add(layers.Dense(714, activation='relu', activity_regularizer=regularizers.l1(0.01) )) # experiment with different activation functions
network.add(layers.Dense(714, activation='relu')) # experiment with different activation functions

network.add(layers.Dropout(0.5))

network.add(layers.Dense(2, activation='softmax'))

In [ ]:
network.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy', metrics=['accuracy', 'categorical_accuracy'])

In [ ]:
network.fit(train_passengers, train_labels, epochs=100, batch_size=8)


In [ ]:
results = network.predict(test_passengers)

In [ ]:
results.shape

In [ ]:
survived = []
for result in results:
    survived.append(result[1])
  

plt.scatter(survived, range(len(survived)))
plt.show()


In [ ]:
# we need binary survival prediction, not a probability
binary_results = network.predict_classes(test_passengers)

In [ ]:
df_test = pd.read_csv("test.csv") # get unwrangled version of test set with the passenger IDs

In [ ]:
df_test['Survived'] = binary_results


In [ ]:
df_test.drop(columns=['Pclass', 'Embarked', 'Name', 'Ticket', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Cabin', 'Fare', 'Embarked'], inplace=True)
    

In [ ]:
df_test.to_csv("predictions.csv", columns = ['PassengerId', 'Survived'])

In [ ]:
df_test.reset_index(drop=True, inplace=True)



In [ ]:
df_test

### TODO
get validation set
get sucess of model on train and validation set with plot, to make sure not overfitting

make a load of synthetic features, like 'is female and lower class'

rescale data

use hyperparam grid search

regularise model

logisitc regression? SVM

cross_validate